In [1]:

# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'dcef29dd-a66a-4378-b28c-f5b1dd177eb1'
resource_group = 'sravia'
workspace_name = 'blahblah'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='diabetes')
df = dataset.to_pandas_dataframe()
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
# let's check if there are any null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
# Learning from our previous dataset, now we know importance of data normalization

# if we don't apply data normalization, large values will always impact the weights more and small values will be ignored

# now let's apply a different type of normalization called z-score 

# formula for z-score: (data - mean) / standard_deviation

# z-score is used for two things:
# a) normalizing the data such that the mean value becomes 0, and all other values are adjusted with 
# respect to mean and standard deviation 
# b) it can also be used to find outliers. If the result of normalization is greater than 3 or less than -3, 
# we can safely assume that it's not normal, and a valid outlier

In [2]:
# even though we have an inbuilt method just like min-max, we can also implment normalization without them
# first, we need the statistics on the data
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [3]:
# we can see above that there is mean and standard deviation present, but they are aligned as rows
# we also know that column selection is much easier than row selection 
# so we can transpose the results so that rows become columns, and columns become row
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [4]:
# now we can easily select mean and standard deviation
# But we also have to be careful that we only do this to features
# outcome is a label, and we should not normalize it

y = df['Outcome']
x = df.iloc[:, :-1]

stats = x.describe().T
mu = stats['mean']
sigma = stats['std']

In [6]:
# now let's apply the formula->.  z-score = (data-mean)/standard_deviation

x_norm = (x-mu)/sigma 
x_norm.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639530,0.847771,0.149543,0.906679,-0.692439,0.203880,0.468187,1.425067
1,-0.844335,-1.122665,-0.160441,0.530556,-0.692439,-0.683976,-0.364823,-0.190548
2,1.233077,1.942458,-0.263769,-1.287373,-0.692439,-1.102537,0.604004,-0.105515
3,-0.844335,-0.997558,-0.160441,0.154433,0.123221,-0.493721,-0.920163,-1.040871
4,-1.141108,0.503727,-1.503707,0.906679,0.765337,1.408828,5.481337,-0.020483


In [8]:
# let's also detect outliers in each column
for column in x_norm.columns:
    # Count outliers
    outlier_count = x_norm[(x_norm[column] > 3) | (x_norm[column] < -3)].shape[0]
    
    # Display the count of outliers
    print(f"Count of outliers in column {column}: {outlier_count}")

Count of outliers in column Pregnancies: 4
Count of outliers in column Glucose: 5
Count of outliers in column BloodPressure: 35
Count of outliers in column SkinThickness: 1
Count of outliers in column Insulin: 18
Count of outliers in column BMI: 14
Count of outliers in column DiabetesPedigreeFunction: 11
Count of outliers in column Age: 5


In [9]:
# one way to manage outliers is that we could have just set them to max or min values!

for column in x_norm.columns:
    # Find the max and min values for the column
    column_max = x_norm[column][x_norm[column] <= 3].max()
    column_min = x_norm[column][x_norm[column] >= -3].min()
    
    # Replace values greater than 3 with the column's max value
    x_norm.loc[x_norm[column] > 3, column] = column_max
    
    # Replace values less than -3 with the column's min value
    x_norm.loc[x_norm[column] < -3, column] = column_min

In [10]:
x_norm.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639530,0.847771,0.149543,0.906679,-0.692439,0.203880,0.468187,1.425067
1,-0.844335,-1.122665,-0.160441,0.530556,-0.692439,-0.683976,-0.364823,-0.190548
2,1.233077,1.942458,-0.263769,-1.287373,-0.692439,-1.102537,0.604004,-0.105515
3,-0.844335,-0.997558,-0.160441,0.154433,0.123221,-0.493721,-0.920163,-1.040871
4,-1.141108,0.503727,-1.503707,0.906679,0.765337,1.408828,2.985325,-0.020483


In [12]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_norm, y, test_size=0.2, random_state=42)
# This parameter, random_state ensures that the random splits of data into training and testing sets are reproducible. 
# When you specify a fixed value for random_state, you ensure that each time you run your code, 
# you get the exact same train/test split of the data.
# this also ensures that when you are trying multiple models multiple times, they have same training and testing data
# random state can be any number greater than or equal to 0, it cannot be negative 


In [25]:
# Now that normalization is done, we can try a classification model, which is really not different from regression
# both regression and classification draw a boundary to fit your data
# regression gives you y = mx + c, while in classification the fitting of mx+ c is used as a threshold 
from sklearn.linear_model import LogisticRegression
# don't go by name- Linear and Logistic Regressions are both classification and regression algorithms depending on usage
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression()

# Fit the model to the training data
model.fit(xtrain, ytrain)

# Make predictions on the test set
predictions = model.predict(xtest)

# Calculate the accuracy of the model
accuracy = accuracy_score(ytest, predictions)
print("Accuracy: " + str(accuracy))

# Optionally, print a detailed classification report
report = classification_report(ytest, predictions, output_dict=True)


Accuracy: 0.7597402597402597


In [28]:
print(report['0']) # performance for non-diabetic predictions

{'precision': 0.81, 'recall': 0.8181818181818182, 'f1-score': 0.8140703517587939, 'support': 99}


In [29]:
print(report['1']) # performance for diabetic predictions

{'precision': 0.6666666666666666, 'recall': 0.6545454545454545, 'f1-score': 0.6605504587155963, 'support': 55}


In [30]:
#Precision is the ratio of correctly predicted positive observations to the total predicted positives. 
#It indicates the quality of the positive class predictions.

#Recall (or sensitivity) is the ratio of correctly predicted positive observations to all the observations in the actual class. 
# It measures the ability of the classifier to find all the positive samples.

#F1-Score is the weighted average of Precision and Recall. 
# It takes both false positives and false negatives into account. 
#It is a better measure than accuracy for imbalanced classes.

#Support refers to the number of actual occurrences of the class in the specified dataset. 
#For example, support: 55 means that there are 55 actual instances of this particular class 



In [33]:
# but how about other algorithms?
# Let's try multiple algorithms and see if any one of them is better than logistic regression 
# we will cover these models in theory later

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [34]:
model_dictionary = {
    'DTree' : DecisionTreeClassifier(),
    'NaiveBayes' : GaussianNB(),
    'RandomForest': RandomForestClassifier(),
    'KNN' : KNeighborsClassifier()
}


In [37]:
# we will also need some metrics to evaluate the models 
accuracies = dict()
f1_scores = dict()


In [39]:
from sklearn.metrics import f1_score

for model_name, model in model_dictionary.items():
    print('Trying model : ' + model_name)
    model.fit(xtrain, ytrain)
    p = model.predict(xtest)
    accuracy = accuracy_score(ytest, p)
    accuracies[model_name] = accuracy
    f1 = f1_score(ytest, p)
    f1_scores[model_name] = f1
    print('Accuracy = ' + str(accuracy))
    print('F1-Score = ' + str(f1))
    print('***********************')
    print()
    

Trying model : DTree
Accuracy = 0.7532467532467533
F1-Score = 0.6724137931034484
***********************

Trying model : NaiveBayes
Accuracy = 0.7402597402597403
F1-Score = 0.6610169491525425
***********************

Trying model : RandomForest
Accuracy = 0.7467532467532467
F1-Score = 0.6486486486486486
***********************

Trying model : KNN
Accuracy = 0.7142857142857143
F1-Score = 0.5686274509803921
***********************



In [40]:
accuracies

{'DTree': 0.7532467532467533,
 'NaiveBayes': 0.7402597402597403,
 'RandomForest': 0.7467532467532467,
 'KNN': 0.7142857142857143}

In [41]:
f1_scores

{'DTree': 0.6724137931034484,
 'NaiveBayes': 0.6610169491525425,
 'RandomForest': 0.6486486486486486,
 'KNN': 0.5686274509803921}

In [46]:
# So which model has the best F1-Score and Accuracy?
# All our answers could be different- every machine has a different learning!
# And - sometimes linear models are just better than the others. 
# There is no way of just saying that one algorithm is better than the other till we train and view the metrics